Breast Cancer Wisconsin (Diagnostic) Data Set

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt2
import matplotlib.cm as cm
%matplotlib inline
from sklearn import preprocessing
from subprocess import check_output

#
print(check_output(["ls", "../input"]).decode("utf8"))





data.csv



In [2]:
# Read the data file
data = pd.read_csv('../input/data.csv')
data.head()


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
# Cleaning and modifying the data
data = data.drop('id',axis=1)
data = data.drop('Unnamed: 32',axis=1)
# Mapping Benign to 0 and Malignant to 1 
data['diagnosis'] = data['diagnosis'].map({'M':1,'B':0})
# Scaling the dataset
datas = pd.DataFrame(preprocessing.scale(data.iloc[:,1:32]))
datas.columns = list(data.iloc[:,1:32].columns)
datas['diagnosis'] = data['diagnosis']
# Creating the high dimensional feature space X
data_drop = datas.drop('diagnosis',axis=1)
X = data_drop.values



In [4]:
# Create a feed forward neural network with 3 hidden layers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input 
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(128,activation="relu",input_dim = np.shape(X)[1]))
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit and test the model by randomly splitting it 
# 67% of the data for training and 33% of the data for validation
model.fit(X, datas['diagnosis'], batch_size=5, epochs=10,validation_split=0.33)




Using TensorFlow backend.


Train on 381 samples, validate on 188 samples
Epoch 1/10
381/381 [==============================] - 0s - loss: 0.2694 - acc: 0.8845 - val_loss: 0.1108 - val_acc: 0.9521
Epoch 2/10
381/381 [==============================] - 0s - loss: 0.1088 - acc: 0.9738 - val_loss: 0.0728 - val_acc: 0.9787
Epoch 3/10
381/381 [==============================] - 0s - loss: 0.0845 - acc: 0.9659 - val_loss: 0.1053 - val_acc: 0.9681
Epoch 4/10
381/381 [==============================] - 0s - loss: 0.0670 - acc: 0.9764 - val_loss: 0.0517 - val_acc: 0.9787
Epoch 5/10
381/381 [==============================] - 0s - loss: 0.0544 - acc: 0.9843 - val_loss: 0.0684 - val_acc: 0.9787
Epoch 6/10
381/381 [==============================] - 0s - loss: 0.0967 - acc: 0.9711 - val_loss: 0.1367 - val_acc: 0.9415
Epoch 7/10
381/381 [==============================] - 0s - loss: 0.0643 - acc: 0.9764 - val_loss: 0.0410 - val_acc: 0.9840
Epoch 8/10
381/381 [==============================] - 0s - loss: 0.0530 - acc: 0.9790 - val_l

In [5]:
# Cross validation analysis 
from sklearn.model_selection import StratifiedKFold
# fix random seed for reproducibility
seed = 3
np.random.seed(seed)
# K fold cross validation (k=2)
k = 2
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
cvscores = []
Y = datas['diagnosis']
for train, test in kfold.split(X, Y):
    # Fit the model
    model.fit(X[train], Y[train], epochs=10, batch_size=10, verbose=0)
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=0)
    # Print scores from each cross validation run 
    # print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%d-fold cross validation accuracy -  %.2f%% (+/- %.2f%%)" % (k,np.mean(cvscores), np.std(cvscores)))

2-fold cross validation accuracy -  98.59% (+/- 0.35%)
